## DATA PREPROCESSING

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [17]:
df = pd.read_excel("category.xlsx",sheet_name= [1,4])
loc_to_cat = df[1]
loc_to_cat['category1'] = loc_to_cat['category1'].str.lower()
allUserData = df[4]
#userData = [df[3], df[4], df[5], df[6], df[7]]

#for i in userData[1:]:
#    allUserData = allUserData.append(i)
allUserData.head()

,user_id,weekday,date,start_time,end_date,end_time,duration,category,category_id,location_name,location_category,my_place,location_id,place_id
0,2,2,2020-06-29,16:00:00,2020-06-29,19:00:00,03:00:00,meeting,13,路易莎,cafe,NaN,4.0,NaN
1,2,2,2020-06-29,19:00:00,2020-06-29,20:00:00,01:00:00,food,1,全哥鵝肉小炒,restaurant,NaN,5.0,NaN
2,2,2,2020-06-29,20:00:00,2020-06-29,20:30:00,00:30:00,leisure,10,NaN,NaN,home,NaN,1.0
3,2,2,2020-06-29,20:30:00,2020-06-29,21:30:00,01:00:00,shopping,12,Miramar,Shopping mall,NaN,6.0,NaN
4,2,2,2020-06-29,21:30:00,2020-06-29,23:30:00,02:00:00,study,5,NaN,NaN,home,NaN,1.0


In [18]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
#drop commute
indexNames = allUserData[(allUserData['category_id'] == 17) ].index
allUserData.drop(indexNames,inplace=True)

allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','end_date','end_time','location_name','location_category','my_place','location_id','place_id'], 1)
allUserData = allUserData.reset_index(drop = True)

In [19]:
def get_hours(ts):
    a = ts.hour
    return a

def get_mins(ts):
    a = ts.minute
    return a

def get_duration(ts):
    a = ts.hour*60 + ts.minute
    return a 

#allUserData['start_mins'] = allUserData['start_time'].apply(get_mins)
allUserData['start_time'] = allUserData['start_time'].apply(get_duration)
allUserData['duration'] = allUserData['duration'].apply(get_duration)
allUserData = allUserData[['user_id','weekday','start_time','duration','location','category','category_id']]
allUserData.tail()

,user_id,weekday,start_time,duration,location,category,category_id
444,2,5,50,495,1,sleep,3
445,2,5,545,15,1,leisure,10
446,2,5,570,150,2,work,6
447,2,5,780,360,3,meeting,13
448,2,5,1200,240,1,leisure,10


## Random Forest Model

In [28]:
train_df = allUserData.loc[0:299]
test_df = allUserData.loc[300:]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 300
Number of testing data is : 149


In [29]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4]].values
y_test = test_df.iloc[:, 6].values

In [30]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [31]:
clf.score(X_test, y_test)

0.6308724832214765

In [32]:
imp = clf.feature_importances_
imp

array([0.        , 0.11707604, 0.48461265, 0.3983113 ])

In [26]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.36      0.71      0.48        21
           3       1.00      0.95      0.97        19
           4       0.00      0.00      0.00         0
           6       0.50      0.82      0.62        11
           9       0.80      0.80      0.80         5
          10       0.80      0.71      0.75        58
          11       0.50      0.12      0.20         8
          12       0.44      0.25      0.32        16
          13       1.00      0.33      0.50         6
          16       0.00      0.00      0.00         2
          18       0.00      0.00      0.00         3

   micro avg       0.63      0.63      0.63       149
   macro avg       0.49      0.43      0.42       149
weighted avg       0.67      0.63      0.62       149



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [27]:
print(confusion_matrix(y_test,predictions))

[[15  0  0  0  0  4  1  1  0  0  0]
 [ 0 18  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  1  0  0  0  1  0]
 [ 0  0  0  0  4  1  0  0  0  0  0]
 [ 8  0  1  5  0 41  0  3  0  0  0]
 [ 5  0  0  0  0  2  1  0  0  0  0]
 [10  0  0  0  1  1  0  4  0  0  0]
 [ 0  0  0  4  0  0  0  0  2  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0]]


In [33]:
predictions

array([12,  1,  1, 10, 10, 10,  3, 10, 10,  1, 10, 11,  1,  1,  4, 10,  3,
       10,  6, 10, 10, 10, 10,  1,  6,  1, 12, 10, 10,  3,  6,  6, 10,  9,
       10,  3,  6,  1,  1, 13, 10, 10,  9, 10,  3,  6,  6,  1, 10, 10,  3,
       10,  1,  1, 12,  1,  1, 11,  1, 10,  3, 10,  3, 10, 12, 12,  1,  6,
        1,  1, 12,  1, 10,  3, 10,  6,  1, 10, 12,  9, 10,  3,  6,  6,  1,
       10,  9, 10, 10,  3, 10,  6,  1, 13,  1,  1,  1, 10, 10,  3, 10,  1,
       16,  1, 10,  3, 10,  1,  1, 10,  1,  1,  1, 10,  3, 10,  1, 10, 12,
       10,  3, 10,  6,  1, 12, 10,  3, 10,  1,  6,  1, 10,  3,  6,  6,  1,
        1,  1,  1,  1,  1, 10,  9, 10,  3, 10,  6,  6, 10])

## User Data

In [3]:
location = pd.read_csv("0830.csv")
location.tail()

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
533,534,121.216483,24.963834,2020-08-28,22:46,6,0.0,-1.0,Guangming Park,藏 Hide Bar,Zhongli Civil Sports Center,中壢國民運動中心全適達泳裝專賣店,中壢國民運動中心籃球場,tourist_attraction,bar,point_of_interest,stadium,point_of_interest
534,535,121.216791,24.964725,2020-08-28,22:50,6,0.0,-1.0,藏 Hide Bar,Hotel Kuva Chateau,Zhongli Civil Sports Center,Guangming Park,中壢國民運動中心全適達泳裝專賣店,bar,lodging,point_of_interest,tourist_attraction,stadium
535,536,121.216725,24.965684,2020-08-28,22:52,6,0.0,-1.0,亞美豆漿大王,Hilife,火燒島日式平價居酒屋,WoodHouse 木宅餐館,天成當舖 24H 汽機車黃金鑽石借款 快速放款 工商融資 房屋二胎 勞力士名錶借款,restaurant,convenience_store,restaurant,restaurant,finance
536,537,121.216725,24.964598,2020-08-28,23:00,6,0.0,-1.0,藏 Hide Bar,Guangming Park,中壢國民運動中心全適達泳裝專賣店,Zhongli Civil Sports Center,Hotel Kuva Chateau,bar,tourist_attraction,stadium,point_of_interest,lodging
537,538,25.087458,25.087458,2020-08-30,20:01,1,0.0,0.0,mo家,4,NaN,NaN,NaN,Home,entering,NaN,NaN,NaN


In [4]:
stopLoc = location.drop(['category3','category4','category5'], 1)

def string_to_mins(ts):
    a = int(ts.split(':')[0])
    b = int(ts.split(':')[1])
    c = a*60 + b
    return c

#stopLoc['start_time'] = pd.to_datetime(stopLoc['start_time'], format='%H:%M', errors='ignore').apply(string_to_mins)
stopLoc['longitude'] = round(stopLoc['longitude'],3)
stopLoc['latitude'] = round(stopLoc['latitude'],3)
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1,c2
3,4,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
4,5,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
5,6,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
6,7,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
7,8,121.574,25.080,2020-08-27,11:37,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,STARBUCKS COFFEE,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),The Executive Centre - Neihu New Century Build...,gym,school
8,9,121.574,25.080,2020-08-27,11:37,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,STARBUCKS COFFEE,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),The Executive Centre - Neihu New Century Build...,gym,school
9,10,121.574,25.080,2020-08-27,11:37,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,STARBUCKS COFFEE,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),The Executive Centre - Neihu New Century Build...,gym,school


In [5]:
for i in range(10):
    dropList = []
    for row in range(1,len(stopLoc)-1): 
        if stopLoc.iloc[row-1,14] == 'entering' and stopLoc.iloc[row,14] != 'exiting':
            dropList.append(row)
            
    print(dropList)
    stopLoc = stopLoc.drop(dropList)
    stopLoc = stopLoc.reset_index(drop = True)
    
stopLoc    

[39, 138, 143, 147, 177, 312]
[39, 137, 141, 144, 173, 307]
[39, 139, 141, 169, 302]
[39, 138, 139, 166, 298]
[39, 137, 294]
[39, 136, 292]
[39, 135, 290]
[39, 134, 288]
[39, 133, 286]
[39, 132, 284]


,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1,c2
3,4,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
4,5,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
5,6,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
6,7,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
7,8,121.574,25.080,2020-08-27,11:37,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,STARBUCKS COFFEE,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),The Executive Centre - Neihu New Century Build...,gym,school
8,9,121.574,25.080,2020-08-27,11:37,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,STARBUCKS COFFEE,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),The Executive Centre - Neihu New Century Build...,gym,school
9,10,121.574,25.080,2020-08-27,11:37,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,STARBUCKS COFFEE,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),The Executive Centre - Neihu New Century Build...,gym,school


In [6]:
#delete trivial data
for i in range(3):
    dropList = []
    for row in range(1,len(stopLoc)-1): 
        if stopLoc.iloc[row,1] == stopLoc.iloc[row-1,1] and stopLoc.iloc[row,2] == stopLoc.iloc[row-1,2]:
            if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
                dropList.append(row)
    
        #compared with the last record, over 2 place name duplicated will be deleted
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        if len(res) > 1: 
            if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
                dropList.append(row)
                
    stopLoc = stopLoc.drop(dropList)
    stopLoc = stopLoc.reset_index(drop = True)
    
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1,c2
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1,c2
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1,c2
3,4,121.574,25.080,2020-08-27,11:13,5,0.0,-1.000000,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,新世紀企業總部大樓,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school
4,11,121.569,25.082,2020-08-27,12:09,5,0.0,-1.000000,馬可先生健康烘焙內湖店,全美自助餐店,合點壽司 がってん寿司 內湖店,洋酒城 內湖門市,萬兩食堂,bakery,restaurant
5,13,121.568,25.082,2020-08-27,12:09,5,0.0,-1.000000,Xihu,7-ELEVEN,芳琳滷肉飯,Xihu Market,中華電信 內湖西湖,subway_station,convenience_store
6,14,121.566,25.082,2020-08-27,12:10,5,0.0,-1.000000,McDonald's,Xihu Market,（烤）小蒙牛頂級麻辣養生鍋Hotpot-內湖店,西雅圖極品咖啡 西湖店,CTBC Bank Sihu Branch,restaurant,point_of_interest
7,15,121.564,25.083,2020-08-27,12:10,5,0.0,-1.000000,Sure Gas Station,帝一帝王蟹吃到飽,Xikang Park,密室逃脫今日有場【玩笑實驗室】Wanxiao Lab,LUXGEN内湖旗舰生活馆,gas_station,restaurant
8,16,121.562,25.084,2020-08-27,12:11,5,0.0,-1.000000,十五郎 日式.洋食,宣捷生物科技股份有限公司,慧誠智醫,iSKI滑雪俱樂部,宣捷幹細胞生技,restaurant,point_of_interest
9,17,121.561,25.085,2020-08-27,12:11,5,0.0,-1.000000,TeaTop台灣第一味台北內湖店,夯麵坊西湖店,來來豆漿,打貓鵝肉亭,7-ELEVEN,point_of_interest,restaurant


In [7]:
#calculate duration
import datetime as dt
from datetime import datetime

for i in range(3):
    dropList = []
    for row in range(0,len(stopLoc)-1): 
        
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        if len(res) > 1: 
            if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
                dropList.append(row)

        #calculate duration
        if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
            stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')       
        else:
            a = dt.datetime(2000,1,2,int(stopLoc.iloc[row+1,4].split(':')[0]),int(stopLoc.iloc[row+1,4].split(':')[1]))
            b = dt.datetime(2000,1,1,int(stopLoc.iloc[row,4].split(':')[0]),int(stopLoc.iloc[row,4].split(':')[1]))
            stopLoc.iloc[row,6] = a - b
            
        #kill duration <10
        if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
            if stopLoc.iloc[row,14] != 'entering' and stopLoc.iloc[row,14] != 'exiting':
                dropList.append(row)     
            
    stopLoc = stopLoc.drop(dropList)
    stopLoc = stopLoc.reset_index(drop = True)
    print(dropList)

stopLoc = stopLoc.drop(['start_date','name2','name3','name4','name5','speed'], 1)
stopLoc

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 80, 84, 87, 88, 89, 90, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 211, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 227, 228, 229, 231, 232, 236, 237, 238]
[]
[]


,location_id,longitude,latitude,start_time,weekday,duration,name1,category1,category2
0,1,121.576,24.986,11:00,4,"1 day, 1:00:00",myhome,c1,c2
1,2,121.443,24.433,12:00,5,"1 day, 1:00:00",apple,c1,c2
2,3,121.123,24.765,13:00\n,6,22:13:00,banana,c1,c2
3,4,121.574,25.080,11:13,5,1:08:00,UFC GYM 內科模範館,gym,school
4,39,25.060,25.060,12:21,5,0:28:00,mo學校,Others,entering
5,138,24.987,24.987,12:49,5,0:06:00,mo學校,School,entering
6,141,24.987,24.987,12:55,5,0:01:00,mo學校,School,exiting
7,143,24.987,24.987,12:56,5,3:19:00,mo學校,School,entering
8,158,121.577,24.986,16:15,5,3:08:00,National Chengchi University,university,university
9,174,121.577,24.987,19:23,5,0:06:00,mo宿舍,Dorm,exiting


## Map Category

In [ ]:
stopLoc = pd.merge(stopLoc, loc_to_cat, how='left', on='category1' )
stopLoc = stopLoc.drop(['name1','category1','location_id'],1)
stopLoc['user_id'] = 2
stopLoc = stopLoc[['user_id','weekday','start_time','duration','location']]
stopLoc
#把start_date加回來
#name1,category1拿掉 place_cat對表換成數字

In [ ]:
traing = stopLoc.iloc[:, np.r_[0:4]].values
predictions = clf.predict(traing)

## Myplace

In [ ]:
savedplace = pd.read_csv("savedPlace.csv")
savedplace['place_longitude'] = round(saveplace['place_longitude'],3)
savedplace['place_latitude'] = round(saveplace['place_latitude'],3)
#savedplace = savedplace.loc[savedplace['my_place'] == 1]
savedplace = savedplace.drop(['place_name','my_place','place_id'],1)
savedplace = savedplace.rename(columns={"place_longitude": "longitude", "place_latitude": "latitude"})
savedplace = savedplace.drop_duplicates(subset=['longitude', 'latitude'], keep='last')
savedplace.head()

In [ ]:
stopLoc = pd.merge(stopLoc, savedplace, how='left', on=['longitude','latitude'])
stopLoc